<a href="https://colab.research.google.com/github/hsuanlion/data-course-sample/blob/main/Rule_based_%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 09:45:37--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  23.2MB/s    in 0.6s    

2021-12-26 09:45:37 (23.2 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-26 09:45:37--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.6MB/s    in 0.5s    

202

In [142]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [143]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [144]:
metadata.shape

(32892, 19)

In [145]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


In [146]:
ratings.shape

(371345, 4)

## 資料整理

In [147]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [148]:
ratings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


## 資料切分

答案：每個user對商品的評分紀錄

In [149]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [132]:
# 一個人可能會有多個評論
ratings_testings.groupby('reviewerID').size().sort_values()

reviewerID
A100XQFWKQ30O2    1
A3F6V5A2SFES04    1
A3FDEHXOYLD1AC    1
A3FGDP5PT4SPLK    1
A3FGWJLDNTJWED    1
                 ..
A2XTIQTNDTY6IA    2
ANR73TPVBCQ9K     2
A1HBXMUIHTAIU2    2
A3VXSKW11C3301    2
A3LW4UWD5IH59X    3
Length: 584, dtype: int64

In [80]:
# 看前11筆資料
dict(list(ratings_testings_by_user.items())[0:11])

{'A100XQFWKQ30O2': ['B01DKQAXC0'],
 'A103T1QOGFCSEH': ['B01DKQAXC0'],
 'A106UKKSJ2KXPF': ['B01ENS6XDC'],
 'A10A7GV4D5A11V': ['B01BFSNF66'],
 'A1119JJ37ZLB8R': ['B01CJNZKZK'],
 'A113UOOLBSZN52': ['B01EKY0M22'],
 'A12M4U7WK4ALCR': ['B01DKQAXC0'],
 'A12T8YTW6VWT7S': ['B01G53GFKO'],
 'A1364JXGKB46MM': ['B01GLA54SA'],
 'A137DALOQFKBTI': ['B01AWXGD3M'],
 'A13FEZ3WV7S2EY': ['B01BHN3EHE']}

In [140]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


## 合併資料

In [159]:
# 合併ratings & metadata_filtered
metadata['rank_n'] = pd.to_numeric(metadata['rank'].str.split('in',expand= True)[0].str.replace(',',''))

metadata_filterd = metadata[['asin', 'brand', 'title', 'price', 'rank', 'description','also_view','also_buy','rank_n']].copy()

metadata_filterd = metadata_filterd.set_index('asin')
metadata_filterd['review_counts'] = ratings.groupby('asin').size()
metadata_filterd['overall_mean'] = ratings.groupby('asin')['overall'].mean()
metadata_filterd['review_counts_train'] = ratings_trainings.groupby('asin').size()
metadata_filterd['overall_mean_train'] = ratings_trainings.groupby('asin')['overall'].mean()
metadata_filterd

,brand,title,price,rank,description,also_view,also_buy,rank_n,review_counts,overall_mean,review_counts_train,overall_mean_train
asin,,,,,,,,,,,,
6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,"2,938,573 in Beauty & Personal Care (",[Loud 'N Clear Personal Sound Amplifier allows...,[],[],2938573.0,2.0,2.5,2.0,2.5
7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,"872,854 in Beauty & Personal Care (",[No7 Lift & Luminate Triple Action Serum 50ml ...,[],"[B01E7LCSL6, B008X5RVME]",872854.0,1.0,3.0,1.0,3.0
7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,$28.76,"956,696 in Beauty & Personal Care (",[No7 Stay Perfect Foundation now stays perfect...,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[],956696.0,1.0,5.0,1.0,5.0
7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,,"1,870,258 in Beauty & Personal Care (",[],[],[B0041PBXX8],1870258.0,1.0,5.0,1.0,5.0
7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,$12.15,"67,701 in Beauty & Personal Care (",[Lacto Calamine Skin Balance Daily Nourishing ...,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[],67701.0,15.0,4.4,15.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...
B01HIWLLUK,,"Barielle Pro Textured Grip Cuticle Nipper, Purple",$9.95,"2,145,325 in Beauty & Personal Care (",[],[],[],2145325.0,1.0,5.0,1.0,5.0
B01HJ1K3YK,Salon Perfect,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,,"1,639,713 in Beauty & Personal Care (",[],[],[],1639713.0,1.0,1.0,1.0,1.0
B01HJ84SGM,,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,$55.63,"207,410 in Beauty & Personal Care (",[],"[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...","[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",207410.0,5.0,5.0,5.0,5.0


## 產生推薦

In [160]:
metadata_filterd.reset_index().sort_values('rank_n',ascending = True).drop_duplicates(subset = ['asin']).head(10)

,asin,brand,title,price,rank,description,also_view,also_buy,rank_n,review_counts,overall_mean,review_counts_train,overall_mean_train
8540,B00BXPVIT8,Braun,"Braun Clean &amp; Renew Refill Cartridges CCR,...",$21.44,35 in Beauty & Personal Care (,"[With Braun Clean & Renew refills, you can kee...","[B07L9PW8R3, B000KND8IS, B0786ZZQK6, B071KCNQT...",[],35.0,13.0,4.846154,13.0,4.846154
6268,B006IB5T4W,Aquaphor,"Aquaphor Healing Ointment,Advanced Therapy Ski...",$11.29,65 in Beauty & Personal Care (,"[Trusted by dermatologists, pediatricians and ...","[B005UEB96K, B07HDDRBX8, B00E3H3260, B00005339...",[],65.0,2790.0,4.643369,2790.0,4.643369
12028,B00HYEPVG8,Astra,Astra Platinum Double Edge Safety Razor Blades...,$9.59,245 in Beauty & Personal Care (,[ASTRA stainless double edge blades fit all do...,"[B00DQ2W7UK, B00PKHIDRA, B07MB4LZC4, B07CGMTVM...",[],245.0,34.0,4.588235,34.0,4.588235
3606,B001QY8QXM,Astra,Astra Platinum Double Edge Safety Razor Blades...,$9.30,280 in Beauty & Personal Care (,[ASTRA stainless double edge blades fit all do...,"[B00DQ2W7UK, B00PKHIDRA, B07CGMTVM4, B07MB4LZC...",[],280.0,4544.0,4.556118,4544.0,4.556118
481,B00005JS5C,Panasonic,Panasonic Bikini Shaper and Trimmer for Women ...,$18.49,457 in Beauty & Personal Care (,"[Perfect for sensitive skin types, this bikini...","[B00TYSQXIC, B0018A32XS, B000G0GW6K, B01IX3M3J...",[],457.0,2249.0,4.088039,2249.0,4.088039
6630,B007EITOSK,Super Sunnies,Super Sunnies Evo Flex Flexible - Random Color...,$3.90,575 in Beauty & Personal Care (,[New Design for the legendary Super Sunnies pr...,[],"[B0058E3XJI, B009YKK78C, B00147FGJ8, B00GTZ6E9...",575.0,831.0,3.904934,831.0,3.904934
8542,B00BXR0SFG,Braun,Braun Clean &amp; Renew Refill Cartridges CCR ...,$11.94,853 in Beauty & Personal Care (,"[With Braun Clean & Renew refills, you can kee...","[B07L9PW8R3, B000KND8IS, B0786ZZQK6, B071KCNQT...",[],853.0,2.0,4.500000,2.0,4.500000
4137,B002OITLG0,Fairy Tales,Fairy Tales Tangle Tamer Static Free &nbsp;De...,$8.40,872 in Beauty & Personal Care (,[This spray favorite detangling spray instantl...,"[B07GFG7D7P, B01GSL9SPC, B01I9TMQOQ, B07G6L8SF...",[],872.0,476.0,4.313025,476.0,4.313025
29823,B01DJI7796,Mother Nature's Essentials,Vitamin E Oil 100% Natural &amp; Organic Highe...,$6.95,"1,017 in Beauty & Personal Care (",[Trying to find a NATURAL ANSWER to REDUCE WRI...,[],"[B01LYMZK4V, B01A74442I, B009U9JUK8, B00987FWH...",1017.0,123.0,4.544715,120.0,4.583333
439,B000050FDY,Braun,Braun Clean &amp; Renew Refill Cartridges CCR ...,$11.94,"1,028 in Beauty & Personal Care (","[With Braun Clean & Renew refills, you can kee...","[B000KND8IS, B07L9PW8R3, B071KCNQTR, B0786ZZQK...",[],1028.0,1538.0,4.651495,1538.0,4.651495


In [161]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    ratings_trainings = training_data
    # default(沒有推進系統)
    # recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    
    # 每人都推薦2018-09月以前，平均分數前10名商品
    # recommendations = {user: metadata_filterd.reset_index().sort_values('overall_mean_train',ascending = False).drop_duplicates(subset = ['asin']).head(10)['asin'].to_list() for user in users}

    # 每人都推薦2018-09月以前，評論次數前10名商品
    recommendations = {user: metadata_filterd.reset_index().sort_values('review_counts_train',ascending = False).drop_duplicates(subset = ['asin']).head(10)['asin'].to_list() for user in users}

    # 每人都推薦有史以來，銷售排名前10名商品
    # recommendations = {user: metadata_filterd.reset_index().sort_values('rank_n',ascending = True).drop_duplicates(subset = ['asin']).head(10)['asin'].to_list() for user in users}

    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007EITOSK',
  'B00BXR0SFG',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY'],
 'A103T1QOGFCSEH': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007EITOSK',
  'B00BXR0SFG',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY'],
 'A106UKKSJ2KXPF': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007EITOSK',
  'B00BXR0SFG',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY'],
 'A10A7GV4D5A11V': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007EITOSK',
  'B00BXR0SFG',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY'],
 'A1119JJ37ZLB8R': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007EITOSK',
  'B00BXR0SFG',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY'],
 'A113UOOLBSZN52': ['B00BXPVIT8',
  'B006IB5T4W',
  'B00HYEPVG8',
  'B001QY8QXM',
  'B00005JS5C',
  'B007

## 結果評估

In [162]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.005084745762711864

In [163]:
# [score records]
# 隨機推薦10個商品
# 0.006779661016949152

# 每人都推薦2018-09月以前，平均分數前10名商品
# 0

# 每人都推薦2018-09月以前，評論次數前10名商品
# 0.08305084745762711

# 每人都推薦有史以來，銷售排名前10名商品
# 0.005084745762711864



In [164]:

0.0831/0.0068 - 1 

11.220588235294118